# 📈 Advanced Stock Analysis Report Generator

A comprehensive multi-agent system for generating detailed investment analysis reports using AI and financial data.

## Features
- 🔍 **Multi-Agent Analysis**: Stock Analyst, Research Analyst, and Investment Lead
- 📊 **Comprehensive Data**: Company info, analyst recommendations, news, and stock prices
- 🎯 **Risk Assessment**: Detailed risk evaluation and market analysis
- 📋 **Portfolio Strategy**: Professional investment recommendations
- 💾 **Report Generation**: Structured markdown reports with executive summary
- ⚡ **Interactive Interface**: Configurable parameters and real-time feedback

## Requirements
- Python 3.10+
- Aideck framework
- YFinance tools
- OpenAI API key (for LLM analysis)
- Internet connection (for real-time data)

## 1. Environment Setup

Load environment variables and configure the system.

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
import warnings
from textwrap import dedent
import json

# Load environment variables
load_dotenv()

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"🚀 Aideck Stock Analysis System")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Project Root: {project_root}")
print(f"✅ Environment loaded successfully!")

## 2. Configuration

Configure the analysis parameters and settings.

In [ ]:
# Configuration class for better organization
class AnalysisConfig:
    def __init__(self):
        # Companies to analyze (comma-separated ticker symbols)
        self.companies = "AAPL,MSFT,GOOGL,AMZN,TSLA"
        
        # Analysis settings
        self.max_news_items = 5
        self.include_price_history = True
        self.include_analyst_recommendations = True
        self.include_company_news = True
        
        # Output settings
        self.generate_charts = False  # Set to True if matplotlib is available
        self.save_reports = True
        self.report_format = "markdown"  # Options: markdown, html, json
        
        # Risk assessment settings
        self.risk_tolerance = "moderate"  # conservative, moderate, aggressive
        self.investment_horizon = "medium"  # short, medium, long
        
    def validate(self):
        """Validate configuration settings"""
        if not self.companies.strip():
            raise ValueError("At least one company must be specified")
            
        valid_tickers = [ticker.strip() for ticker in self.companies.split(',') if ticker.strip()]
        if len(valid_tickers) == 0:
            raise ValueError("No valid company tickers found")
            
        self.companies = ", ".join(valid_tickers)
        print(f"✅ Configuration validated for {len(valid_tickers)} companies: {self.companies}")
        
    def display(self):
        """Display current configuration"""
        print("\n📋 Analysis Configuration:")
        print(f"   Companies: {self.companies}")
        print(f"   Max News Items: {self.max_news_items}")
        print(f"   Include Price History: {self.include_price_history}")
        print(f"   Include Analyst Recommendations: {self.include_analyst_recommendations}")
        print(f"   Include Company News: {self.include_company_news}")
        print(f"   Risk Tolerance: {self.risk_tolerance}")
        print(f"   Investment Horizon: {self.investment_horizon}")
        print(f"   Generate Charts: {self.generate_charts}")
        print(f"   Save Reports: {self.save_reports}")
        print(f"   Report Format: {self.report_format}")

# Create and validate configuration
config = AnalysisConfig()
config.validate()
config.display()

## 3. Import Dependencies

Import all required modules with error handling.

In [ ]:
def safe_import(module_name, description):
    """Safely import a module with error handling"""
    try:
        module = __import__(module_name)
        print(f"✅ {description}: {module_name}")
        return module
    except ImportError as e:
        print(f"❌ Failed to import {description}: {module_name}")
        print(f"   Error: {e}")
        return None

# Import core dependencies
print("\n📦 Importing Dependencies...")

from aideck import Agent, Task
print("✅ Core Aideck modules imported")

# Import YFinance tools
try:
    from aideck.tools import YFinanceTools
    print("✅ YFinance tools imported")
except ImportError:
    print("❌ YFinance tools not available. Please install: pip install yfinance")
    print("   Continuing without YFinance tools...")
    YFinanceTools = None

# Optional imports for enhanced functionality
matplotlib = safe_import('matplotlib', 'Data visualization')
plotly = safe_import('plotly', 'Interactive charts')
pandas = safe_import('pandas', 'Data analysis')
numpy = safe_import('numpy', 'Numerical computing')

print(f"\n🎯 Ready to analyze {len(config.companies.split(', '))} companies!")

## 4. Initialize Tools and Agents

Set up the YFinance tools and create specialized agents.

In [ ]:
def initialize_tools(config):
    """Initialize YFinance tools with configuration"""
    if YFinanceTools is None:
        print("❌ YFinance tools not available")
        return None
    
    try:
        tools = YFinanceTools(
            company_info=True,
            analyst_recommendations=config.include_analyst_recommendations,
            company_news=config.include_company_news,
            stock_price=config.include_price_history,
            enable_all=True
        )
        print(f"✅ YFinance tools initialized with {len(tools.functions())} functions")
        return tools
    except Exception as e:
        print(f"❌ Error initializing YFinance tools: {e}")
        return None

def create_agents():
    """Create specialized analysis agents"""
    try:
        # Stock Analyst - Focuses on data collection and basic analysis
        stock_analyst = Agent(
            name="Stock Analyst",
            role="Senior Market Analyst at Goldman Sachs",
            goal="Collect and analyze comprehensive market data for investment decisions",
            backstory="""You are MarketMaster-X, an elite Senior Investment Analyst with 15+ years of experience
            at Goldman Sachs. You specialize in comprehensive market analysis, financial statement evaluation,
            industry trend identification, and news impact assessment. Your expertise includes:
            - Deep financial analysis and ratio interpretation
            - Market trend and competitive positioning analysis
            - News sentiment and impact evaluation
            - Risk factor identification and assessment
            - Growth catalyst and opportunity identification"""
        )
        
        # Research Analyst - Focuses on comparative analysis and ranking
        research_analyst = Agent(
            name="Research Analyst",
            role="Senior Research Analyst at Goldman Sachs",
            goal="Provide comparative analysis and investment potential ranking",
            backstory="""You are ValuePro-X, an elite Senior Research Analyst specializing in investment
            opportunity evaluation, comparative analysis, and risk-reward assessment. Your expertise includes:
            - Investment potential evaluation and ranking
            - Comparative valuation analysis
            - Competitive advantage assessment
            - Risk-adjusted return analysis
            - Strategic positioning evaluation
            - Growth sustainability analysis"""
        )
        
        # Investment Lead - Focuses on portfolio strategy and recommendations
        investment_lead = Agent(
            name="Investment Lead",
            role="Senior Investment Lead at Goldman Sachs",
            goal="Develop portfolio allocation strategies and final investment recommendations",
            backstory="""You are PortfolioSage-X, a distinguished Senior Investment Lead with expertise in
            portfolio strategy development, asset allocation optimization, and client recommendation delivery.
            Your expertise includes:
            - Portfolio strategy and asset allocation
            - Risk management and diversification
            - Investment rationale articulation
            - Client presentation and communication
            - Strategic investment planning
            - Performance optimization"""
        )
        
        print("✅ All agents created successfully")
        return stock_analyst, research_analyst, investment_lead
        
    except Exception as e:
        print(f"❌ Error creating agents: {e}")
        return None, None, None

# Initialize tools and agents
yfinance_tools = initialize_tools(config)
stock_analyst, research_analyst, investment_lead = create_agents()

if yfinance_tools and stock_analyst and research_analyst and investment_lead:
    print("\n🎯 System ready for analysis!")
else:
    print("\n⚠️  System partially initialized. Some features may not work.")

## 5. Market Analysis

Perform comprehensive market analysis using the Stock Analyst agent.

In [ ]:
def perform_market_analysis(stock_analyst, yfinance_tools, config):
    """Perform comprehensive market analysis"""
    if not all([stock_analyst, yfinance_tools]):
        print("❌ Cannot perform analysis: missing required components")
        return None
    
    print(f"\n📊 Starting Market Analysis for: {config.companies}")
    print("=" * 60)
    
    # Create analysis task
    analysis_task = Task(
        description=dedent(f"""\
        You are MarketMaster-X, an elite Senior Investment Analyst at Goldman Sachs with expertise in:
        - Comprehensive market analysis
        - Financial statement evaluation
        - Industry trend identification
        - News impact assessment
        - Risk factor analysis
        - Growth potential evaluation

        Instructions:
        1. Market Research 📊
           - Analyze company fundamentals and metrics
           - Review recent market performance
           - Evaluate competitive positioning
           - Assess industry trends and dynamics
        2. Financial Analysis 💹
           - Examine key financial ratios
           - Review analyst recommendations
           - Analyze recent news impact
           - Identify growth catalysts
        3. Risk Assessment 🎯
           - Evaluate market risks
           - Assess company-specific challenges
           - Consider macroeconomic factors
           - Identify potential red flags
        
        **Important Notes:**
        - This analysis is for educational purposes only
        - Always consult with qualified financial advisors
        - Consider your risk tolerance: {config.risk_tolerance}
        - Investment horizon: {config.investment_horizon}
        - Use real-time data when available

        Analyze the following companies and produce a comprehensive market analysis report in markdown format: {config.companies}
        """),
        tools=yfinance_tools.functions()
    )
    
    try:
        print("🤖 Stock Analyst is analyzing the market data...")
        market_report = stock_analyst.do(analysis_task)
        print("✅ Market analysis completed successfully!")
        return market_report
        
    except Exception as e:
        print(f"❌ Error during market analysis: {e}")
        return None

# Perform the analysis
market_report = perform_market_analysis(stock_analyst, yfinance_tools, config)

## 6. Research Analysis

Perform comparative analysis and investment ranking using the Research Analyst.

In [ ]:
def perform_research_analysis(research_analyst, market_report, config):
    """Perform comparative research analysis and ranking"""
    if not research_analyst or not market_report:
        print("❌ Cannot perform research analysis: missing required components")
        return None
    
    print("\n🔍 Starting Research Analysis and Ranking")
    print("=" * 60)
    
    # Create research task
    research_task = Task(
        description=dedent(f"""\
        You are ValuePro-X, an elite Senior Research Analyst at Goldman Sachs specializing in:
        - Investment opportunity evaluation
        - Comparative analysis
        - Risk-reward assessment
        - Growth potential ranking
        - Strategic recommendations

        Instructions:
        1. Investment Analysis 🔍
           - Evaluate each company's potential
           - Compare relative valuations
           - Assess competitive advantages
           - Consider market positioning
        2. Risk Evaluation 📈
           - Analyze risk factors
           - Consider market conditions
           - Evaluate growth sustainability
           - Assess management capability
        3. Company Ranking 🏆
           - Rank based on investment potential
           - Provide detailed rationale
           - Consider risk-adjusted returns
           - Explain competitive advantages

        **Analysis Context:**
        - Risk Tolerance: {config.risk_tolerance}
        - Investment Horizon: {config.investment_horizon}
        - Focus on long-term value creation
        - Consider both quantitative and qualitative factors

        Based on the following market analysis, rank the companies by investment potential and provide a detailed investment analysis and ranking report in markdown format:

        {market_report}
        """),
        tools=[]  # No external tools needed for this analysis
    )
    
    try:
        print("🤖 Research Analyst is evaluating investment opportunities...")
        research_report = research_analyst.do(research_task)
        print("✅ Research analysis completed successfully!")
        return research_report
        
    except Exception as e:
        print(f"❌ Error during research analysis: {e}")
        return None

# Perform the research analysis
research_report = perform_research_analysis(research_analyst, market_report, config)

## 7. Portfolio Strategy

Develop portfolio allocation strategy using the Investment Lead.

In [ ]:
def develop_portfolio_strategy(investment_lead, research_report, config):
    """Develop comprehensive portfolio allocation strategy"""
    if not investment_lead or not research_report:
        print("❌ Cannot develop portfolio strategy: missing required components")
        return None
    
    print("\n💼 Developing Portfolio Strategy")
    print("=" * 60)
    
    # Create portfolio strategy task
    portfolio_task = Task(
        description=dedent(f"""\
        You are PortfolioSage-X, a distinguished Senior Investment Lead at Goldman Sachs expert in:
        - Portfolio strategy development
        - Asset allocation optimization
        - Risk management
        - Investment rationale articulation
        - Client recommendation delivery

        Instructions:
        1. Portfolio Strategy 💼
           - Develop allocation strategy
           - Optimize risk-reward balance
           - Consider diversification
           - Set investment timeframes
        2. Investment Rationale 📝
           - Explain allocation decisions
           - Support with analysis
           - Address potential concerns
           - Highlight growth catalysts
        3. Recommendation Delivery 📊
           - Present clear allocations
           - Explain investment thesis
           - Provide actionable insights
           - Include risk considerations

        **Portfolio Context:**
        - Risk Tolerance: {config.risk_tolerance}
        - Investment Horizon: {config.investment_horizon}
        - Diversification requirements
        - Risk management priorities
        - Performance expectations

        Based on the following investment analysis and ranking, develop a portfolio allocation strategy and provide a final investment report in markdown format:

        {research_report}
        """),
        tools=[]  # No external tools needed for strategy development
    )
    
    try:
        print("🤖 Investment Lead is developing portfolio strategy...")
        portfolio_report = investment_lead.do(portfolio_task)
        print("✅ Portfolio strategy completed successfully!")
        return portfolio_report
        
    except Exception as e:
        print(f"❌ Error during portfolio strategy development: {e}")
        return None

# Develop portfolio strategy
portfolio_report = develop_portfolio_strategy(investment_lead, research_report, config)

## 8. Generate Comprehensive Report

Combine all reports into a comprehensive investment analysis document.

In [ ]:
def generate_comprehensive_report(market_report, research_report, portfolio_report, config):
    """Generate a comprehensive investment report"""
    if not all([market_report, research_report, portfolio_report]):
        print("❌ Cannot generate report: missing required reports")
        return None
    
    print("\n📋 Generating Comprehensive Investment Report")
    print("=" * 60)
    
    # Generate timestamp for report
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Create executive summary
    executive_summary = f"""# Executive Summary

**Analysis Date:** {timestamp}
**Companies Analyzed:** {config.companies}
**Risk Tolerance:** {config.risk_tolerance.capitalize()}
**Investment Horizon:** {config.investment_horizon.capitalize()}

## Key Insights
- Comprehensive market analysis completed for {len(config.companies.split(', '))} companies
- Detailed risk assessment and competitive positioning analysis
- Investment potential ranking based on multiple factors
- Strategic portfolio allocation recommendations
- Professional-grade investment rationale and recommendations

## Important Disclaimers
- This analysis is for educational and informational purposes only
- Not intended as financial advice or investment recommendations
- Always consult with qualified financial advisors before making investment decisions
- Market conditions can change rapidly and affect investment outcomes
- Past performance does not guarantee future results

---
"""
    
    # Combine all reports
    comprehensive_report = (
        f"# 📈 Comprehensive Investment Analysis Report\n\n"
        f"**Generated by Aideck Multi-Agent System** | **Date:** {timestamp}\n\n"
        f"---\n\n"
        f"{executive_summary}\n"
        f"# 1. Market Analysis Report\n\n"
        f"{market_report}\n\n"
        f"---\n\n"
        f"# 2. Research Analysis & Ranking\n\n"
        f"{research_report}\n\n"
        f"---\n\n"
        f"# 3. Portfolio Strategy & Recommendations\n\n"
        f"{portfolio_report}\n\n"
        f"---\n\n"
        f"# 📊 Report Metadata\n\n"
        f"**Analysis Parameters:**\n"
        f"- Companies: {config.companies}\n"
        f"- Risk Tolerance: {config.risk_tolerance}\n"
        f"- Investment Horizon: {config.investment_horizon}\n"
        f"- Report Format: {config.report_format}\n"
        f"- Generated: {timestamp}\n\n"
        f"**System Information:**\n"
        f"- Aideck Multi-Agent Analysis System\n"
        f"- Powered by AI and Real-time Market Data\n"
        f"- Professional-grade Investment Analysis\n"
    )
    
    return comprehensive_report

def save_report(report, config):
    """Save the comprehensive report to file"""
    if not config.save_reports or not report:
        return
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    companies_short = config.companies.replace(', ', '_')[:50]
    filename = f"investment_report_{companies_short}_{timestamp}.{config.report_format}"
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report)
        print(f"✅ Report saved successfully: {filename}")
        
        # Also save as JSON for structured data
        json_filename = f"investment_report_{companies_short}_{timestamp}.json"
        report_data = {
            "timestamp": timestamp,
            "companies": config.companies,
            "config": {
                "risk_tolerance": config.risk_tolerance,
                "investment_horizon": config.investment_horizon,
                "report_format": config.report_format
            },
            "reports": {
                "market_analysis": market_report,
                "research_analysis": research_report,
                "portfolio_strategy": portfolio_report
            },
            "comprehensive_report": report
        }
        
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(report_data, f, indent=2, ensure_ascii=False)
        print(f"✅ Structured data saved: {json_filename}")
        
    except Exception as e:
        print(f"❌ Error saving report: {e}")

# Generate and save the comprehensive report
comprehensive_report = generate_comprehensive_report(market_report, research_report, portfolio_report, config)

if comprehensive_report:
    print("\n📄 Displaying Comprehensive Report:")
    print("=" * 80)
    print(comprehensive_report)
    print("=" * 80)
    
    # Save the report
    save_report(comprehensive_report, config)
else:
    print("❌ Failed to generate comprehensive report")

## 9. Analysis Summary

Display a summary of the completed analysis.

In [ ]:
def display_analysis_summary(config, market_report, research_report, portfolio_report):
    """Display a summary of the completed analysis"""
    print("\n🎯 Analysis Summary")
    print("=" * 50)
    
    # Check completion status
    market_complete = market_report is not None
    research_complete = research_report is not None
    portfolio_complete = portfolio_report is not None
    
    print(f"📊 Market Analysis: {'✅ Complete' if market_complete else '❌ Failed'}")
    print(f"🔍 Research Analysis: {'✅ Complete' if research_complete else '❌ Failed'}")
    print(f"💼 Portfolio Strategy: {'✅ Complete' if portfolio_complete else '❌ Failed'}")
    
    if all([market_complete, research_complete, portfolio_complete]):
        print("\n🎉 All analysis phases completed successfully!")
        
        # Calculate approximate report length
        total_length = len(market_report) + len(research_report) + len(portfolio_report)
        print(f"📏 Total report length: ~{total_length:,} characters")
        
        print(f"\n📋 Analysis completed for: {config.companies}")
        print(f"⚙️  Configuration: {config.risk_tolerance} risk, {config.investment_horizon} horizon")
        
        if config.save_reports:
            print("💾 Reports have been saved to files")
        
        print("\n💡 Next Steps:")
        print("   1. Review the comprehensive report above")
        print("   2. Check saved files for detailed analysis")
        print("   3. Consult financial advisors before investing")
        print("   4. Monitor market conditions regularly")
        
    else:
        print("\n⚠️  Some analysis phases failed. Check error messages above.")
        print("\n🔧 Troubleshooting:")
        print("   1. Verify API keys are set in .env file")
        print("   2. Check internet connection for data access")
        print("   3. Ensure all dependencies are installed")
        print("   4. Review error messages for specific issues")

# Display the analysis summary
display_analysis_summary(config, market_report, research_report, portfolio_report)

## 📚 Additional Resources

### Configuration Options
- **Companies**: Modify `config.companies` to analyze different stocks
- **Risk Tolerance**: Set to 'conservative', 'moderate', or 'aggressive'
- **Investment Horizon**: Choose 'short', 'medium', or 'long' term
- **Data Sources**: Configure which financial data to include

### Customization Examples
```python
# Analyze specific tech stocks
config.companies = "NVDA,AMD,INTC,TSM"

# Conservative investment approach
config.risk_tolerance = "conservative"
config.investment_horizon = "long"

# Include price history analysis
config.include_price_history = True
```

### File Outputs
- **Markdown Report**: `investment_report_[companies]_[timestamp].md`
- **JSON Data**: `investment_report_[companies]_[timestamp].json`
- **Structured data** for further analysis or integration

---

**Disclaimer**: This analysis is generated using AI and publicly available data. It is for educational and informational purposes only and should not be considered as financial advice. Always consult with qualified financial advisors and conduct your own due diligence before making investment decisions.